<a href="https://colab.research.google.com/github/leon0702/Co-teachingIMDb/blob/main/CoteachingIMDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
from google.colab import drive
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
from keras.preprocessing import sequence
from keras import backend as K
import sys
import time
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
def bce(x,y):
  b = tf.keras.losses.BinaryCrossentropy(from_logits = False, reduction = tf.keras.losses.Reduction.NONE)
  loss = b(y,x)
  return loss
def colossA(x1, y1, x2, y2, BS, FR):
  loss_1 = bce(y1,x1)
  loss_1_sortedtemp, loss_1_ind = tf.nn.top_k(-loss_1, len(x2))
  loss_1_sorted = loss_1_sortedtemp*-1
  loss_2 = bce(y2,x2)
  loss_2_sortedtemp, loss_2_ind = tf.nn.top_k(-loss_2, len(x2))
  loss_2_sorted = loss_2_sortedtemp*-1
  remember_rate = 1 - FR
  newBS = int(len(x2)*remember_rate)
  a_data = []
  a_dataind = []
  b_data = []
  a_label = []
  b_label = []
  for i in range(0,newBS):
    a_data.append(x2[loss_2_ind[i]])
    b_data.append(x1[loss_1_ind[i]])
    a_dataind.append(loss_2_ind[i])
    a_label.append(y2[loss_2_ind[i]])
    b_label.append(y1[loss_1_ind[i]])
  a_data = np.array(a_data)
  b_data = np.array(b_data)
  a_dataind = np.array(a_dataind)
  a_label = np.array(a_label)
  return  a_dataind, a_label
def colossB(x1, y1, x2, y2, BS, FR):
  loss_1 = bce(y1,x1)
  loss_1_sortedtemp, loss_1_ind = tf.nn.top_k(-loss_1, len(x1))
  loss_1_sorted = loss_1_sortedtemp*-1
  loss_2 = bce(y2,x2)
  loss_2_sortedtemp, loss_2_ind = tf.nn.top_k(-loss_2, len(x1))
  loss_2_sorted = loss_2_sortedtemp*-1
  remember_rate = 1 - FR
  newBS = int(len(x1)*remember_rate)
  a_data = []
  b_dataind = []
  b_data = []
  a_label = []
  b_label = []
  for i in range(0,newBS):
    a_data.append(x2[loss_2_ind[i]])
    b_data.append(x1[loss_1_ind[i]])
    b_dataind.append(loss_1_ind[i])
    a_label.append(y2[loss_2_ind[i]])
    b_label.append(y1[loss_1_ind[i]])
  a_data = np.array(a_data)
  b_data = np.array(b_data)
  b_dataind = np.array(b_dataind)
  a_label = np.array(a_label)
  b_label = np.array(b_label)
  """
  print("colossB")
  print("FR =",FR)
  print("Remember_rate =",remember_rate)
  print("b_dataind =",b_dataind)
  """
  return  b_dataind, b_label
def trainA(dataa,labela,labelb,BS,EP,logitsa,logitsb):
  
  newAdata_ind, newAlabels = colossA(logitsa,labela,logitsb,labelb,BS,rate_schedule[EP])
  #print("newAdata_ind",newAdata_ind)
  with tf.GradientTape() as tape:
    logits = Onea(dataa[newAdata_ind],training = True)
    loss_value = bce(logits, newAlabels)

  trainable_varsA = Onea.trainable_variables
  gradients = tape.gradient(loss_value, trainable_varsA)
  optimizer.apply_gradients(zip(gradients, trainable_varsA))

  A_loss(loss_value)
  for metric in A_accuracy:
    metric(newAlabels, logits)
  print_status_bar(i*BS,len(One_partial_x_train),A_loss,A_accuracy)
  return  A_loss.result(), metric.result()
def trainB(labela,datab,labelb,BS,EP,logitsa,logitsb):
  newBdata_ind, newBlabels = colossB(logitsa,labela,logitsb,labelb,BS,rate_schedule[EP])
  #print("newBdata_ind",newBdata_ind)
  with tf.GradientTape() as t:
    logits = Oneb(datab[newBdata_ind],training = True)
    loss_value = bce(logits, newBlabels)

  trainable_varsB = Oneb.trainable_variables
  gradients = t.gradient(loss_value, trainable_varsB)
  optimizer.apply_gradients(zip(gradients, trainable_varsB))

  B_loss(loss_value)
  for metric in B_accuracy:
    metric(newBlabels, logits)
  print_status_bar(i*BS,len(One_partial_x_train),B_loss,B_accuracy)
  return  B_loss.result(), metric.result()
def valA(data,label):
  logita = Onea(data, training = False)
  loss_value = bce(logita, label)
  A_val.update_state(label, logita)
  A_val_loss.update_state(loss_value)
  return A_val.result(), A_val_loss.result()
def valB(data,label):
  logitb = Oneb(data, training = False)
  loss_value = bce(logitb, label)
  B_val.update_state(label, logitb)
  B_val_loss.update_state(loss_value)
  return B_val.result(), B_val_loss.result()
def progress_bar(iteration, total, size=30):
  running = iteration < total
  c = ">" if running else "="
  p = (size - 1) * iteration // total
  fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
  params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
  return fmt.format(*params)
def print_status_bar(iteration, total, loss, metrics=None, size=30):
  metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                      for m in [loss] + (metrics or [])])
  end = "" if iteration < total else "\n"
  print("\r{} - {}".format(progress_bar(iteration, total),""" metrics"""), end=end)
def computeEnsemble(xnow,EP,templogit):
    #templogit = templogit*(1-(alpha*(EP/EPOCH)))+xnow*(alpha*(EP/EPOCH))
    templogit = (templogit*(1-alpha)+xnow*alpha)
    return templogit
def find_noise(y,ensx,tempx):
  a_dataind = []
  a_label = []
  tempind = []
  noisecount = 0
  cleancount = 0
  tempclean = 0
  tempnoise = 0
  #print("Ensemble accuracy = ",tf.keras.metrics.BinaryAccuracy()(y,ensx))
  #print("Start Check and Count")
  for i in range(0,len(ensx)):
    #觀察ensemble的預測結果
    if ensx[i]>=0.5:
        prelab = 1.0
    else:
        prelab = 0.0
    if prelab == y[i]:
        cleancount +=1
        #a_dataind.append(i)
        a_label.append(y[i])
    else:
        noisecount +=1
        
        a_dataind.append(i)
        """
        if y[i] == 1.0:
            a_label.append(1.0-y[i])
        else:
            a_label.append(1.0+y[i])
        """
        #print("Maybe Noise Number",i," = ",ensx[i])
    #觀察目前為止的預測結果
    if tempx[i]>=0.5:
        templab = 1.0
    else:
        templab = 0.0
    if templab == y[i]:
        tempclean +=1
    else:
        tempnoise +=1
        tempind.append(i)
  a_dataind = np.array(a_dataind)
  tempind = np.array(tempind)
  a_label = np.array(a_label)
  print("\nEnsemble Noise number = ",noisecount)
  print("Ensemble Clean number = ",cleancount)
  print("Now Noise number = ",tempnoise)
  print("Now Clean number = ",tempclean)
  return  a_dataind, a_label,tempind


In [11]:
#-----0% to 40%
#--------------------------------------讀訓練資料
data_dir = '/content/drive/My Drive/Colab Notebooks/IMDb/0%noise'
train_dir = os.path.join(data_dir, 'train_1.txt')

f= open(train_dir, encoding='UTF-8')
train_data = f.read()
f.close()

train_data = train_data.split('\n')
train_data= train_data[:-1]

labels = []
train_texts = []
ind = []
for index, item in enumerate(train_data):
    ind.append(index)
    label=[x for x in item.split(',')]
    text=label[3:]
    text=" ".join(text)
    train_texts.append(text)
    if label[2]=='1':
        labels.append(1)
    else:
        labels.append(0)
#-------------------------------------------------------------------------------------------------------------------讀順序
data_dir = '/content/drive/My Drive/Colab Notebooks/IMDb/0%noise'
train_dir = os.path.join(data_dir, 'A_train.txt')

f= open(train_dir, encoding='UTF-8')
train = f.read()
f.close()

train = train.split('\n')
train= train[:-1]
#--------------------------------------讀測試訓練資料
data_dir = '/content/drive/My Drive/Colab Notebooks/IMDb/0%noise'
fname = os.path.join(data_dir, 'test_1.txt')

f= open(fname, encoding='UTF-8')
test = f.read()
f.close()

test = test.split('\n')
test= test[:-1]

y_labels=[]
test_texts=[]

for index, item in enumerate(test):
    label=[x for x in item.split(',')]
    text=label[3:]
    text=" ".join(text)
    test_texts.append(text)
    if label[2]=='1':
        y_labels.append(1)
    else:
        y_labels.append(0)

#--------------------------------------------轉訓練資料順序 要先轉順序才可以加noise

train_texts_1=[]
labels_1 =[]
for indexs in range(len(train)):
    number = int(train[indexs])
    train_texts_1.append(train_texts[number])
    labels_1.append(labels[number])
train_texts = train_texts_1
labels = labels_1
clean_labels = labels.copy()
texts = train_texts+test_texts

#------------------------------------------------------------------------------------------------------------讀noise
data_dir = '/content/drive/My Drive/Colab Notebooks/IMDb/40%noise'
noise_dir = os.path.join(data_dir, 'A_noise.txt')

f= open(noise_dir, encoding='UTF-8')
noise = f.read()
f.close()

noise = noise.split('\n')
noise= noise[:-1]
co = []
valco = []
#------------------------------------------------------------加入noise

for indexes in range(len(noise)):
    number = int(noise[indexes])
    if  number >=2000:
        co.append(number)
        if labels[number]>=1:
            labels[number]=0
        else:
            labels[number]=1
    elif number<2000:
        valco.append(number)

print(len(co),len(valco))

#-------------------------------------------------前處理Tokenzier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
maxlen = 1000  # We will cut reviews after 1200 words

tokenizer = Tokenizer(num_words=10000, lower=True, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',oov_token = 'unk')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
#---------------------------------------------分train, test data
x_train=data[:25000]
x_test=data[25000:]

y_train = np.zeros((len(labels),1))
y_train = np.asarray(y_train).astype('float32')
y_test = np.zeros((len(y_labels),1))
y_test = np.asarray(y_test).astype('float32')
#----------------------------------------------label
for i in range(0,(len(labels))):
  if labels[i] == 1.0:
    y_train[i] = 1.0
for i in range(0,len(y_labels)):
  if y_labels[i] == 1.0:
    y_test[i] = 1.0
#----------------------------------------------------切資料
x_val = x_train[:2000]
partial_x_train = x_train[2000:]
#partial_x_train = np.append(pxt,pxt2,axis = 0)

y_val = y_train[:2000]
partial_y_train = y_train[2000:]
#partial_y_train = np.append(pyt,pyt2)
max_features=10000
#-------------------------------------------------------One hot

import numpy as np


def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

One_x_train = vectorize_sequences(x_train)# 整筆訓練資料
One_partial_x_train = vectorize_sequences(partial_x_train) #One hot 
One_x_val = vectorize_sequences(x_val)# One-hot 輸入的驗證資料
One_x_test = vectorize_sequences(x_test)# One-hot 輸入的測試資料

9198 802


In [12]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
A_loss = tf.keras.metrics.Mean()
B_loss = tf.keras.metrics.Mean()
A_accuracy = [tf.keras.metrics.BinaryAccuracy(threshold=0.5)]
B_accuracy = [tf.keras.metrics.BinaryAccuracy(threshold=0.5)]
A_val_loss = tf.keras.metrics.Mean()
B_val_loss = tf.keras.metrics.Mean()
A_val = tf.keras.metrics.BinaryAccuracy()
B_val = tf.keras.metrics.BinaryAccuracy()
# Keep results for plotting
train_loss_resultsA = []
train_accuracy_resultsA = []
train_loss_resultsB = []
train_accuracy_resultsB = []
val_loss_resultsA = []
val_accuracy_resultsA = []
val_loss_resultsB = []
val_accuracy_resultsB = []
EPOCH = 30
BS = 512
forget_rate = 0.4
alpha = 1.0
exponent = 1 #exponent of the forget rate. This parameter is equal to c in Tc for R(T) in Co-teaching paper.
num_gradual = EPOCH+1 #how many epochs for linear drop rate. This parameter is equal to Tk for R(T) in Co-teaching paper.
rate_schedule = np.ones(EPOCH+1)*forget_rate
rate_schedule[:num_gradual] = np.linspace(0, forget_rate**exponent, num_gradual)

#-----------------------One Hot
dataA = One_partial_x_train
dataB = One_partial_x_train
labelsA = partial_y_train
labelsB = partial_y_train
val_dataA = One_x_val
val_dataB = One_x_val
val_labelsA = y_val
val_labelsB = y_val

pre_numupdates = int(len(One_partial_x_train)/BS)
numupdates = int(len(One_partial_x_train)/BS)
val_numupdates = int(len(One_x_val)/BS)
print(rate_schedule)
Onea = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation = tf.nn.leaky_relu, input_shape = (10000,)),
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16, activation = tf.nn.leaky_relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)
    ],)
Oneb = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation = tf.nn.leaky_relu, input_shape = (10000,)),
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16, activation = tf.nn.leaky_relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)
    ],)
#Onea = tf.keras.models.load_model('Model/OneaBest',compile = False)
#Oneb = tf.keras.models.load_model('Model/OneaBest',compile = False)
earlycount = 0
tempAval = 100
tempBval = 100
hitlist = []
noiselist = []
nowhitlist = []
nownoiselist = []
y_ensemble = np.zeros((len(partial_y_train),1))
for epoch in range(1,EPOCH+1):
  
  if earlycount >1:
    print("Early Stopping!!")
    break
  
  print("[INFO] starting epoch {}/{}...".format(epoch, EPOCH))
  sys.stdout.flush()
  epochStart = time.time()
  localtime = time.ctime()
  print("Start at", localtime)
  
  for i in range(0,numupdates+1):
    #print("Iteration :",i+1)
    start = i * BS
    end = start + BS
    logitsa = Onea(dataA[start:end],training = False)
    logitsb = Oneb(dataB[start:end],training = False)
    loA, accA = trainA(dataA[start:end],labelsA[start:end],labelsB[start:end],BS,epoch,logitsa,logitsb)
    loB, accB = trainB(labelsA[start:end],dataB[start:end], labelsB[start:end],BS,epoch,logitsa,logitsb)
  train_loss_resultsA.append(loA)
  train_accuracy_resultsA.append(accA)
  train_loss_resultsB.append(loB)
  train_accuracy_resultsB.append(accB)
  for i in range(0,val_numupdates+1):
    start = i * BS
    end = start + BS
    valA_acc, valA_loss = valA(val_dataA[start:end],val_labelsA[start:end])
    valB_acc, valB_loss = valB(val_dataB[start:end],val_labelsB[start:end])
  if ((valA_loss>tempAval) and (valB_loss>tempBval)):
    earlycount = earlycount+1
  elif ((valA_loss<tempAval) and (valB_loss<tempBval)):
    earlycount = 0
  print("\nEarly Stop count",earlycount)
  val_loss_resultsA.append(valA_loss)
  val_accuracy_resultsA.append(valA_acc)
  val_loss_resultsB.append(valB_loss)
  val_accuracy_resultsB.append(valB_acc)
  tempAval = valA_loss
  tempBval = valB_loss
  A_val.reset_states()
  B_val.reset_states()
  A_val_loss.reset_states()
  B_val_loss.reset_states()
  #alpha = max(0.1,(alpha - (alpha/EPOCH)))
  #print("ALPHA",alpha)
  print("Epoch {:d}: Loss A: {:.3f}, Accuracy A: {:.3%}, Loss B: {:.3f}, Accuracy B: {:.3%}".format(epoch, loA, accA, loB, accB))
  print("Epoch {:d}: Val_A_Loss: {:.3f}, Val_A_Accuracy: {:.3%}, Val_B_Loss: {:.3f}, Val_B_Accuracy: {:.3%}".format(epoch, valA_loss, valA_acc, valB_loss, valB_acc))
  # show timing information for the epoch
  for metric in [A_loss]+A_accuracy:
    metric.reset_states()
  for metric in [B_loss]+B_accuracy:
    metric.reset_states()
  
  epochEnd = time.time()
  ltime = time.ctime()
  print("End at", ltime)
  elapsed = (epochEnd - epochStart)
  print("Used {:.4} seconds".format(elapsed))
Onea.compile(
              loss='binary_crossentropy',
							metrics=["acc"])
(lossa, acca) = Onea.evaluate(One_x_test, y_test,batch_size=BS)
print("[INFO] test accuracy: {:.4f}".format(acca))
Oneb.compile(
              loss='binary_crossentropy',
							metrics=["acc"])
(lossb, accb) = Oneb.evaluate(One_x_test, y_test,batch_size=BS)
print("[INFO] test accuracy: {:.4f}".format(accb))

[0.         0.01333333 0.02666667 0.04       0.05333333 0.06666667
 0.08       0.09333333 0.10666667 0.12       0.13333333 0.14666667
 0.16       0.17333333 0.18666667 0.2        0.21333333 0.22666667
 0.24       0.25333333 0.26666667 0.28       0.29333333 0.30666667
 0.32       0.33333333 0.34666667 0.36       0.37333333 0.38666667
 0.4       ]
[INFO] starting epoch 1/30...
Start at Mon Jan 11 11:46:39 2021
22528/23000 [============================>.] -  metrics
Early Stop count 0
Epoch 1: Loss A: 0.692, Accuracy A: 51.611%, Loss B: 0.692, Accuracy B: 51.532%
Epoch 1: Val_A_Loss: 0.684, Val_A_Accuracy: 60.200%, Val_B_Loss: 0.683, Val_B_Accuracy: 60.450%
End at Mon Jan 11 11:47:23 2021
Used 43.61 seconds
[INFO] starting epoch 2/30...
Start at Mon Jan 11 11:47:23 2021
22528/23000 [============================>.] -  metrics
Early Stop count 0
Epoch 2: Loss A: 0.686, Accuracy A: 55.120%, Loss B: 0.685, Accuracy B: 55.255%
Epoch 2: Val_A_Loss: 0.672, Val_A_Accuracy: 62.650%, Val_B_Loss: 0.